In [1]:
import qkeras
import tensorflow as tf
from qkeras.quantizers import quantized_bits
import fkeras as fk
from fkeras.utils import quantize_and_bitflip, quantize_and_bitflip_deterministic
from tabulate import tabulate
import numpy as np
np.random.seed(1029384756)

2023-03-01 18:38:14.928339: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-01 18:38:15.061468: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-01 18:38:15.061492: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-01 18:38:15.867374: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# How Does FKeras Inject Faults?
The purpose of this notebook is to explain the process FKeras uses to inject faults (i.e., bit flips) via the `fkeras.utils.quantize_and_bitflip` function. We'll begin by defining exactly what we mean by "fault injection" in the context of FKeras and then we will work through the process of creating the necessary code/functions to make a generalized `quantize_and_bitflip` function.

## What sort of faults are we injecting? 
In FKeras, a single fault injection maps onto the action of flipping a single bit in one of the parameters (e.g., weights, biases, inputs, etc.) of a Keras/QKeras model. Let's assume we have a QKeras model with a CONV2D layer and we want to inject a single fault into one of the weights in that CONV2D Layer. 

Before we inject the fault, let's review the quantization that QKeras applies to weight matrix. Assume the following is a representation of the CONV2D layer's original (unquantized) weight matrix.

<!-- $$\begin{bmatrix} 0.0 & -2.5 \\ -0.1258 & 1.879\end{bmatrix}$$ -->

In [2]:
weights_u = tf.convert_to_tensor(np.random.rand(2,2,3))
for w in weights_u.numpy():
    print(w)

[[0.42638122 0.0285622  0.14838862]
 [0.61286693 0.7276006  0.31245513]]
[[0.9670236  0.44825718 0.42079282]
 [0.31039814 0.27730493 0.7789492 ]]


2023-03-01 18:38:17.042809: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-01 18:38:17.042854: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-01 18:38:17.042892: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fabricant): /proc/driver/nvidia/version does not exist
2023-03-01 18:38:17.043422: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


QKeras allows users to quantize the weights of a model layer via the use of a QKeras quantizer. The quantizer relies on a fixed-point representation in order to achieve the quantization. The following cell instantiates a QKeras quantizer which will quantize the weights to values that are representable by a fixed-point representation made up of 5 bits with 1 sign-bit, 1 integer bit, and 3 fractional bits. *It should be noted that QKeras will steal a bit from the fractional bits for the sign bit if `keep_negative=1`*.  

In [3]:
test_quantizer_kn1 = quantized_bits(bits=5, integer=1, keep_negative=1, alpha=1)
test_quantizer_kn0 = quantized_bits(bits=5, integer=1, keep_negative=1, alpha=1)

The fixed-point representation defined in the quantizer can represent the following values.

In [4]:
test_quantizer_kn1.range()

array([ 0.   ,  0.125,  0.25 ,  0.375,  0.5  ,  0.625,  0.75 ,  0.875,
        1.   ,  1.125,  1.25 ,  1.375,  1.5  ,  1.625,  1.75 ,  1.875,
       -2.   , -1.875, -1.75 , -1.625, -1.5  , -1.375, -1.25 , -1.125,
       -1.   , -0.875, -0.75 , -0.625, -0.5  , -0.375, -0.25 , -0.125],
      dtype=float32)

If we apply the quantizer to the original weight matrix, we get the following quantized weights.

In [5]:
weights_q = test_quantizer_kn1(weights_u)
for w in weights_q.numpy():
    print(w)

[[0.375 0.    0.125]
 [0.625 0.75  0.25 ]]
[[1.    0.5   0.375]
 [0.25  0.25  0.75 ]]


In [6]:
for i in range(weights_q.numpy().shape[0]):
    for j in range(weights_q.numpy().shape[1]):
        for k in range(weights_q.numpy().shape[2]):
            print(f"Weight {(i,j,k)}: {weights_u.numpy()[i][j][k]:.10f}  -->  {weights_q.numpy()[i][j][k]:.3f}")

Weight (0, 0, 0): 0.4263812245  -->  0.375
Weight (0, 0, 1): 0.0285622043  -->  0.000
Weight (0, 0, 2): 0.1483886153  -->  0.125
Weight (0, 1, 0): 0.6128669344  -->  0.625
Weight (0, 1, 1): 0.7276006002  -->  0.750
Weight (0, 1, 2): 0.3124551335  -->  0.250
Weight (1, 0, 0): 0.9670235976  -->  1.000
Weight (1, 0, 1): 0.4482571768  -->  0.500
Weight (1, 0, 2): 0.4207928178  -->  0.375
Weight (1, 1, 0): 0.3103981442  -->  0.250
Weight (1, 1, 1): 0.2773049277  -->  0.250
Weight (1, 1, 2): 0.7789492049  -->  0.750


The output from the previous cells show that, after the application of the quantizer, each weight in the weight matrix gets mapped to one of the representable values of the quantizer. Since we want to inject faults (i.e., bit flips) after this quantization occurs, we simply need a way of mapping from one of the quantizer's representable values to another.

In [7]:
for i, rv in enumerate(test_quantizer_kn1.range()):
    rv_str = f"{rv:2.3f}"
    print(f" {i:05b} : {rv_str:>6}")

 00000 :  0.000
 00001 :  0.125
 00010 :  0.250
 00011 :  0.375
 00100 :  0.500
 00101 :  0.625
 00110 :  0.750
 00111 :  0.875
 01000 :  1.000
 01001 :  1.125
 01010 :  1.250
 01011 :  1.375
 01100 :  1.500
 01101 :  1.625
 01110 :  1.750
 01111 :  1.875
 10000 : -2.000
 10001 : -1.875
 10010 : -1.750
 10011 : -1.625
 10100 : -1.500
 10101 : -1.375
 10110 : -1.250
 10111 : -1.125
 11000 : -1.000
 11001 : -0.875
 11010 : -0.750
 11011 : -0.625
 11100 : -0.500
 11101 : -0.375
 11110 : -0.250
 11111 : -0.125


In [8]:
quant_config = test_quantizer_kn1.get_config()
scaling_exponent = quant_config["bits"] - quant_config["integer"] - quant_config["keep_negative"]

In [20]:
flbi_regions = [
    fk.utils.FaultyLayerBitRegion(0, 0, 1.0),
    fk.utils.FaultyLayerBitRegion(1, 1, 1.0),
    ]

print(fk.utils.quantize_and_bitflip_deterministic_v3(weights_u*-1, test_quantizer_kn1, []          , []))
print()
print()
print(fk.utils.quantize_and_bitflip_deterministic_v3(weights_u*-1, test_quantizer_kn1, flbi_regions, []))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tf.Tensor(
[[[-0.375  0.    -0.125]
  [-0.625 -0.75  -0.25 ]]

 [[-1.    -0.5   -0.375]
  [-0.25  -0.25  -0.75 ]]], shape=(2, 2, 3), dtype=float32)


array([-8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
tf.Tensor(
[[[ 0.625  0.    -0.125]
  [-0.625 -0.75  -0.25 ]]

 [[-1.    -0.5   -0.375]
  [-0.25  -0.25  -0.75 ]]], shape=(2, 2, 3), dtype=float32)


In [19]:
print(f"{8:05b}")
print(f"{16:05b}")

curr_val = 8

sign_mask = 2**(i_qbits - i_keep_negative)
rval_mask = sign_mask -1
for i in range(mask_array.shape[0]):
    mask_array[i] = (mask_array[i] & rval_mask) - (mask_array[i] & sign_mask)

01000
10000


In [10]:
#0.5 <--> -1.5
a = int(0.5 * 2**(scaling_exponent))
b = int(0b10000)

result = a ^ b

print(f"{a:05b} {b:05b} {result:05b}")

print(f"{a} {b} {result}")

i_tensor * (2**-i_scaling_exp)

00100 10000 10100
4 16 20


NameError: name 'i_tensor' is not defined

In [11]:
(int(0.5 * 2**scaling_exponent) ^ -16)*2**-scaling_exponent

-1.5

In [12]:
0.5

0.5

In [13]:
int(0.5 * 2**scaling_exponent)

4

In [14]:
(int(0.5 * 2**scaling_exponent) ^ int(0b10000))

20

In [15]:
(int(0.5 * 2**scaling_exponent) ^ -int(0b10000))

-12

In [16]:
(int(0.5 * 2**scaling_exponent) ^ int(0b10000))*2**-scaling_exponent

2.5

In [17]:
(int(0.5 * 2**scaling_exponent) ^ -int(0b10000))

-12

In [18]:
(int(0.5 * 2**scaling_exponent) ^ -int(0b10000))*2**-scaling_exponent

-1.5

In [19]:
f"{-16:05b}"

'-10000'

In [20]:
bin(int(-1.5 * 2**(scaling_exponent)))

'-0b1100'

In [21]:
bin(20)

'0b10100'

In [22]:
bin(-12)

'-0b1100'

In [23]:
int(0b11100)

28

In [24]:
    og_dtype = i_tensor.dtype
    i_tensor = i_tensor * (2**i_scaling_exp)
    i_tensor = tf.cast(i_tensor, tf.int64)
    i_tensor = tf.bitwise.bitwise_xor(
        i_tensor, gen_mask_tensor_random(i_tensor, i_ber, i_qbits)
    )
    i_tensor = tf.cast(i_tensor, og_dtype)
    i_tensor = i_tensor * (2**-i_scaling_exp)

NameError: name 'i_tensor' is not defined

In [25]:
def full_tensor_quantize_and_bit_flip_manual(i_tensor, i_scaling_exp, i_ber, i_qbits, i_mask_tensor):
    og_dtype = i_tensor.dtype
    i_tensor = i_tensor * (2**i_scaling_exp)
    i_tensor = tf.cast(i_tensor, tf.int64)
    i_tensor = tf.bitwise.bitwise_xor(
        i_tensor, i_mask_tensor
    )
    i_tensor = tf.cast(i_tensor, og_dtype)
    i_tensor = i_tensor * (2**-i_scaling_exp)

    return i_tensor

manual_mask_tensor = np.zeros((2,2,3), dtype=np.int64)
manual_mask_tensor[0,0,0] = -16
manual_mask_tensor = tf.convert_to_tensor( manual_mask_tensor, dtype=np.int64)

full_tensor_quantize_and_bit_flip_manual(weights_q, scaling_exponent, 1/60, quant_config["bits"], manual_mask_tensor).numpy()

array([[[-1.625,  0.   ,  0.125],
        [ 0.625,  0.75 ,  0.25 ]],

       [[ 1.   ,  0.5  ,  0.375],
        [ 0.25 ,  0.25 ,  0.75 ]]], dtype=float32)

In [26]:
print(weights_q.numpy())
print("####################")
print(fk.utils.full_tensor_quantize_and_bit_flip_deterministic(weights_q, scaling_exponent, 1/60, quant_config["bits"]).numpy())
print("####################")
print(fk.utils.full_tensor_quantize_and_bit_flip_deterministic_v2(weights_q, scaling_exponent, 1/60, quant_config["bits"], quant_config["keep_negative"]).numpy())
print("####################")
print(fk.utils.full_tensor_quantize_and_bit_flip_deterministic_v2(weights_q, scaling_exponent, 1/60, quant_config["bits"], quant_config["keep_negative"]).numpy())

[[[0.375 0.    0.125]
  [0.625 0.75  0.25 ]]

 [[1.    0.5   0.375]
  [0.25  0.25  0.75 ]]]
####################
[[[0.25  0.    0.125]
  [0.625 0.75  0.25 ]]

 [[1.    0.5   0.375]
  [0.25  0.25  0.75 ]]]
####################
[[[0.25  0.    0.125]
  [0.625 0.75  0.25 ]]

 [[1.    0.5   0.375]
  [0.25  0.25  0.75 ]]]
####################
[[[0.25  0.    0.125]
  [0.625 0.75  0.25 ]]

 [[1.    0.5   0.375]
  [0.25  0.25  0.75 ]]]


In [10]:
fk.utils.quantize_and_bitflip_deterministic_v2(weights_u, test_quantizer_kn1, [], [60/60], i_keep_negative=1)

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[-0.390625, -0.015625, -0.140625],
        [-0.640625, -0.765625, -0.265625]],

       [[-1.015625, -0.515625, -0.390625],
        [-0.265625, -0.265625, -0.765625]]], dtype=float32)>

In [11]:
print(fk.utils.gen_mask_tensor_deterministic(weights_q,60/60,quant_config["bits"]))
print(fk.utils.gen_mask_tensor_random(weights_q,60/60,quant_config["bits"]))


NameError: name 'quant_config' is not defined

In [28]:
def gen_mask_tensor_deterministic_v2(i_tensor, i_ber, i_qbits, i_keep_negative=0):
    # S: Generate the mask array (default value is 0)
    mask_array = np.full(i_tensor.shape, 0).flatten()

    # S: Determine the number of bits in region
    num_rbits = mask_array.size * i_qbits

    # S: Determine the number of faults to inject
    num_rfaults = int(num_rbits * i_ber)

    # S: Inject faults
    faults_injected = 0
    while faults_injected < num_rfaults:
        # print(mask_array)
        mask_array[faults_injected % mask_array.size] = mask_array[
            faults_injected % mask_array.size
        ] + 2 ** (faults_injected // mask_array.size)
        faults_injected = faults_injected + 1

    sign_mask = 2**(i_qbits - i_keep_negative)
    rval_mask = sign_mask -1
    for i in range(mask_array.shape[0]):
        mask_array[i] = (mask_array[i] & rval_mask) - (mask_array[i] & sign_mask)

    return tf.convert_to_tensor(np.reshape(mask_array, i_tensor.shape), dtype=tf.int64)

print(fk.utils.gen_mask_tensor_deterministic(weights_q,30/60,quant_config["bits"]))
print(         gen_mask_tensor_deterministic_v2(weights_q,30/60,quant_config["bits"], 1))


tf.Tensor(
[[[7 7 7]
  [7 7 7]]

 [[3 3 3]
  [3 3 3]]], shape=(2, 2, 3), dtype=int64)
tf.Tensor(
[[[7 7 7]
  [7 7 7]]

 [[3 3 3]
  [3 3 3]]], shape=(2, 2, 3), dtype=int64)


In [40]:
# Goal
# I want to be able to flip a specific set of bits in a particular layer
# In order to accomplish this, I need to be able to generate a mask tensor
# that will mask the bits that I want to flip for that layer.

def gen_mask_tensor_deterministic_v3(i_tensor, i_flbirs, i_qbits, i_keep_negative=0):
    # S: Generate the mask array (default value is 0)
    mask_array = np.full(i_tensor.shape, 0).flatten()

    # S: Inject faults
    for curr_flbir in i_flbirs:
        #S: Get the start LBI and end LBI
        s_lbi, e_lbi, rber = (curr_flbir.start_lbi, curr_flbir.end_lbi, curr_flbir.ber)

        #S: Get the weight bit index representation of the LBI
        ### TODO: This code assumes a region of a single bit (s_lbi == e_lbi)
        ### Update this code to be more general.
        s_wbi = fk.utils.lb_index_to_wb_index(s_lbi,i_qbits)
        
        #S: Flip the bit of indicated weight
        mask_array[s_wbi[0]] = mask_array[s_wbi[0]] | (1 << s_wbi[1])


    print(mask_array)

    sign_mask = 2**(i_qbits - i_keep_negative)
    rval_mask = sign_mask - 1
    for i in range(mask_array.shape[0]):
        print(mask_array[i])
        print((mask_array[i] & rval_mask))
        print((mask_array[i] & sign_mask))
        mask_array[i] = (mask_array[i] & rval_mask) - (mask_array[i] & sign_mask)
        print(mask_array[i])
        print()
    print(sign_mask)
    print(mask_array)

    return tf.convert_to_tensor(np.reshape(mask_array, i_tensor.shape), dtype=tf.int64)

print(fk.utils.gen_mask_tensor_deterministic(weights_q,1/60,quant_config["bits"]))
print(         gen_mask_tensor_deterministic_v2(weights_q,1/60,quant_config["bits"], 1))

flbi_regions = [
    fk.utils.FaultyLayerBitRegion(1, 0, 1.0),
    # fk.utils.FaultyLayerBitRegion(1, 1, 1.0),
    ]
print(         gen_mask_tensor_deterministic_v3(weights_q,flbi_regions,quant_config["bits"], 1))


tf.Tensor(
[[[1 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]]], shape=(2, 2, 3), dtype=int64)
tf.Tensor(
[[[1 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]]], shape=(2, 2, 3), dtype=int64)
[8 0 0 0 0 0 0 0 0 0 0 0]
8
8
0
8

0
0
0
0

0
0
0
0

0
0
0
0

0
0
0
0

0
0
0
0

0
0
0
0

0
0
0
0

0
0
0
0

0
0
0
0

0
0
0
0

0
0
0
0

16
[8 0 0 0 0 0 0 0 0 0 0 0]
tf.Tensor(
[[[8 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]]], shape=(2, 2, 3), dtype=int64)


In [36]:
fk.utils.lb_index_to_wb_index(0,quant_config["bits"])

(0, 4)

In [43]:
8 & 16

0

In [48]:
entry_in_mask_array = 0
print(f"{entry_in_mask_array:05b}")
entry_in_mask_array = entry_in_mask_array | 1 << 0
print(f"{entry_in_mask_array:05b}")
entry_in_mask_array = entry_in_mask_array | 1 << 1
print(f"{entry_in_mask_array:05b}")
entry_in_mask_array = entry_in_mask_array | 1 << 2
print(f"{entry_in_mask_array:05b}")
entry_in_mask_array = entry_in_mask_array | 1 << 3
print(f"{entry_in_mask_array:05b}")
entry_in_mask_array = entry_in_mask_array | 1 << 4
print(f"{entry_in_mask_array:05b}")

00000
00001
00011
00011
01011
11011


In [57]:
print(f"{8:05b}")
print(f"{16:05b}")

01000
10000
